Diagonalizing the Hamiltonian matrix for the Rydberg hamiltonian to find the energy eigenvalues and eigenkets. Calculate the groundstate magnetization.

$$
H = -\sum_{\langle i j \rangle} V_{ij} (\sigma^z_i \sigma^z_j + \sigma^z_i + \sigma^z_j) - \Omega \sum_{i=1}^N \sigma^x_i - \frac{h}{2} \sum_{i=1}^N \sigma^z_i - \left(\frac{Nh}{2} + \sum_{\langle i j \rangle} V_{ij}\right)
$$
where ${\bf \sigma}_i$ are Pauli operators and $V_{ij} = \frac{C}{R_{ij}^6} = \frac{1}{r}\frac{C}{|i-j|^6}$ where $r$ is the nearest neighbour length.

In [17]:
using LinearAlgebra

N = 3
Dim = 2^N

r = 1. # this is the nearest neighbour length scale
C = 6.
Ω = 0.5 # this is the transverse field
h = 0.5 # this is the longitudinal field

Hamiltonian = zeros(Dim,Dim)   #This is your 2D Hamiltonian matrix

function Vij(i,j)
    return C / (r*abs(i-j)^6)
end

# Calculate the sum of all Vij's for the diagonal off-set
sumVij = 0
for i = 1:N-1
    for j = i+1:(N)
        sumVij += Vij(i,j)
    end
end    

for Ket = 0:Dim-1  #Loop over Hilbert Space
    Diagonal = 0.
    for SpinIndex = 0:N-2  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        NextIndex = SpinIndex + 1
        Spin2 = 2*((Ket>>NextIndex)&1) - 1
        Diagonal = Diagonal - Vij(SpinIndex,NextIndex)*Spin1*Spin2 - 0.5*h*Spin1 #spins are +1 and -1
    end
    
    # above loop only goes until N-2'th spin
    SpinN = 2*((Ket>>(N-1))&1) - 1
    Diagonal = Diagonal - 0.5*h*SpinN
    # add constant terms
    Diagonal = Diagonal - N*h/2 - sumVij
        
    Hamiltonian[Ket+1,Ket+1] = Diagonal
    
    # transverse field term
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1,Ket+1] = -Ω
    end 
end

println(Hamiltonian)

[-24.09375 -0.5 -0.5 0.0 -0.5 0.0 0.0 0.0; -0.5 -12.59375 0.0 -0.5 0.0 -0.5 0.0 0.0; -0.5 0.0 -0.59375 -0.5 0.0 0.0 -0.5 0.0; 0.0 -0.5 -0.5 -13.09375 0.0 0.0 0.0 -0.5; -0.5 0.0 0.0 0.0 -12.59375 -0.5 -0.5 0.0; 0.0 -0.5 0.0 0.0 -0.5 -1.09375 0.0 -0.5; 0.0 0.0 -0.5 0.0 -0.5 0.0 -13.09375 -0.5; 0.0 0.0 0.0 -0.5 0.0 -0.5 -0.5 -25.59375]


In [18]:
Diag = eigen(Hamiltonian)     #Diagonalize the Hamiltonian
GroundState = Diag.vectors[:, 1];  #this gives the groundstate eigenvector
E0 = Diag.values[1] / N
println(E0)

magnetization = 0
for Ket = 0:Dim-1  #Loop over Hilbert Space
    SumSz = 0.
    for SpinIndex = 0:N-1  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        SumSz += Spin1 #spin is +1 or -1
        #print(Spin1," ")
    end
    #println(SumSz," ",GroundState[Ket+1])
    magnetization += SumSz*SumSz*GroundState[Ket+1]^2  #Don't forgot to square the coefficients...
end
println(magnetization/(N*N))

-8.547994208948127
0.996782333543586
